In [81]:
from dateutil import parser, rrule
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd #DataFrame, Series
import time
import graphviz
import pydotplus
import collections

In [82]:
# import trip data 
trips = pd.read_csv('Bixi_data/OD_2019-06.csv')

#get size of df
num_trips = trips.shape[0]
print(num_trips)

933409


In [83]:
# extract month, day, hour to trips
#convert string to timestamp and imputate seconds
trips.start_date = pd.to_datetime(trips.start_date) - pd.to_timedelta(pd.to_datetime(trips.start_date).dt.second, unit='s')
trips.end_date = pd.to_datetime(trips.end_date) - pd.to_timedelta(pd.to_datetime(trips.end_date).dt.second, unit='s')
#extract month from timestamp
trips['month'] = trips.start_date.dt.month
#extract weekday from timestamp
trips['weekday'] = trips.start_date.dt.weekday
#extract hour from timestamp
trips['hour'] = trips.start_date.dt.hour
trips['round_to_hour'] = trips.start_date.dt.round("H")
#extract date from timestamp
trips['date'] = trips.start_date.dt.date


start_date  start_station_code            end_date  \
0      2019-06-01 00:00:00                6026 2019-06-01 00:03:00   
1      2019-06-01 00:00:00                6197 2019-06-01 00:24:00   
2      2019-06-01 00:00:00                6435 2019-06-01 00:11:00   
3      2019-06-01 00:00:00                7036 2019-06-01 00:21:00   
4      2019-06-01 00:00:00                7003 2019-06-01 00:12:00   
...                    ...                 ...                 ...   
933404 2019-06-30 23:59:00                6015 2019-07-01 00:10:00   
933405 2019-06-30 23:59:00                6012 2019-07-01 00:30:00   
933406 2019-06-30 23:59:00                6119 2019-07-01 00:35:00   
933407 2019-06-30 23:59:00                5007 2019-07-01 00:35:00   
933408 2019-06-30 23:59:00                6408 2019-07-01 00:03:00   

        end_station_code  duration_sec  is_member  month  weekday  hour  \
0                   6036           197          0      6        5     0   
1                   6006 

In [84]:
# import temp data
temp = pd.read_csv('Weather_data/Hourly/en_climate_hourly_QC_702S006_06-2019_P1H.csv')
# Commented out since it takes way too long to compute all the data
# temp = temp.append(pd.read_csv('Weather_data/Hourly/en_climate_hourly_QC_702S006_05-2019_P1H.csv'))
# temp = temp.append(pd.read_csv('Weather_data/Hourly/en_climate_hourly_QC_702S006_06-2019_P1H.csv'))
# temp = temp.append(pd.read_csv('Weather_data/Hourly/en_climate_hourly_QC_702S006_07-2019_P1H.csv'))
# temp = temp.append(pd.read_csv('Weather_data/Hourly/en_climate_hourly_QC_702S006_08-2019_P1H.csv'))
# temp = temp.append(pd.read_csv('Weather_data/Hourly/en_climate_hourly_QC_702S006_09-2019_P1H.csv'))
# temp = temp.append(pd.read_csv('Weather_data/Hourly/en_climate_hourly_QC_702S006_10-2019_P1H.csv'))


In [85]:
# dictionary of temperature data
# concatinate date and time with temperature
temp_df = pd.concat([temp['Date/Time'], temp['Temp (°C)']], axis=1)
temp_df['Date/Time'] = pd.to_datetime(temp_df['Date/Time'])
temp_df = temp_df.set_index('Date/Time').T
# convert df to dictionary
temp_dict = temp_df.to_dict('list')

In [86]:
# import precipitation data
precip = pd.read_csv('Weather_data/Daily/en_climate_daily_QC_702S006_2019_P1D.csv')
# dictionary of rain data
rain_df = pd.concat([precip['Date/Time'], precip['Total Precip (mm)']], axis=1)
rain_df['Date/Time'] = pd.to_datetime(rain_df['Date/Time'])
rain_df['Date/Time'] = rain_df['Date/Time'].dt.date
# fill na with 0
rain_df = rain_df.fillna(0)
# get the transpose of the df to generate dictionary of date and rain
rain_df = rain_df.set_index('Date/Time').T
rain_dict = rain_df.to_dict('list')
# dictionary of snow data
snow_df = pd.concat([precip['Date/Time'], precip['Total Snow (cm)']], axis=1)
snow_df['Date/Time'] = pd.to_datetime(snow_df['Date/Time'])
snow_df['Date/Time'] = snow_df['Date/Time'].dt.date
snow_df = snow_df.fillna(0)
# transpose the df to get dictionary
snow_df = snow_df.set_index('Date/Time').T
snow_dict = snow_df.to_dict('list')
#map weather to trips
trips['Temp'] = trips.round_to_hour.map(temp_dict)
trips['Rain'] = trips.date.map(rain_dict)
trips['Snow'] = trips.date.map(snow_dict)

# change Temp column from object to int and fill NaN values
trips['Temp'] = (trips.Temp.str[0])
trips['Temp'] = pd.to_numeric(trips.Temp , errors='ignore')
trips['Temp'] = trips.Temp.fillna(0)
trips['Temp'] = trips.Temp.astype(int)

# # change Rain column from object to int and fill NaN values
trips['Rain'] = (trips.Rain.str[0])
trips['Rain'] = pd.to_numeric(trips.Rain, errors='ignore')
trips['Rain'] = trips.Rain.fillna(0)
trips['Rain'] = trips.Rain.astype(int)

# change Snow column from object to int and fill NaN values
trips['Snow'] = (trips.Snow.str[0])
trips['Snow'] = pd.to_numeric(trips.Snow, errors='ignore')
trips['Snow'] = trips.Snow.fillna(0)
trips['Snow'] = trips.Snow.astype(int)

In [87]:
# compute trip volumes 
# compute number of bikes checked out
first = min(trips.start_date)
last = max(trips.end_date)
# creates list which has the date for every minute
vol_time = list(rrule.rrule(freq=rrule.MINUTELY, dtstart=first, until=last))
# empty list 
vol_val = np.zeros(len(vol_time))
# creates dictionary for the time and the value
vol_dict = dict(zip(vol_time, vol_val))
# fill volume dictionary
for i in np.arange(num_trips):
    start = trips.start_date.iloc[i]
    end = trips.end_date.iloc[i]
    # check every trips, if a minute belongs to it then increment the occurrance of that minute
    for j in list(rrule.rrule(freq=rrule.MINUTELY, dtstart=start, until=end)):
        vol_dict[j] += 1
trips['volume'] = trips.start_date.map(vol_dict)
volume = trips.volume.values

In [88]:
# Natural Data
nat_df = pd.concat([trips['Temp'], trips['Rain'], trips['Snow'], 
                   trips['hour'], trips['weekday'], trips['month'],
                     trips['volume']], axis=1)
x_df_nat = nat_df.drop(['volume'], axis=1)
x_matrix_nat = x_df_nat.astype(float).values

In [89]:
# engineered data
eng_df = nat_df
# weekday processing 
eng_df.weekday = eng_df.weekday.replace(np.arange(5), 1)
# set weekends to 0
eng_df.weekday = eng_df.weekday.replace(5, 0)
eng_df.weekday = eng_df.weekday.replace(6, 0)
# rain processing
# set to 1 if more than 0 (binary data encoding)
eng_df.loc[eng.Rain > 0] = 1

# snow processing
# set to 1 if more than 0 (binary data encoding)
eng_df.loc[eng.Snow > 0] = 1
#eng['Snow'][eng['Snow'] > 0] = 1

# season processing
# set warm months to 1
eng_df.month = eng_df.month.replace(np.arange(5,10), 1)
# non warm months to 0
eng_df.month = eng_df.month.replace(4, 0)
eng_df.month = eng_df.month.replace(np.arange(10,13), 0)

In [90]:

x_df_eng = eng_df.drop(['volume'], axis=1)
x_matrix_eng = x_df_eng.astype(float).values

In [91]:
np.random.seed(2)
# use 70% of dataset for training, rest for testing
nat_train = int(num_trips*0.70)
# create array size of n_train and get random values from n_trips to fill it
train_id = np.random.choice(num_trips, nat_train, replace=False)
# create test set
test_id = np.array(list(set(range(num_trips))-set(train_id)))
num_test = num_trips - nat_train
# training datasets
x_train_nat = x_matrix_nat[train_id,:]
x_train_eng = x_matrix_eng[train_id,:]
y_train = volume[train_id]
# testing dataset
x_test_nat = x_matrix_nat[test_id,:]
x_test_eng = x_matrix_eng[test_id,:]
y_test = volume[test_id]

In [92]:
# Error Functions
def rmse(predict, test):
    num_test = len(predict)
    error = 1/num_test * np.sqrt(sum((predict - test)**2))
    return error

In [93]:
def percent_error(predict, test):
    error = np.mean(np.divide(np.abs(test-predict),predict))
    return error

In [94]:
def treePlot(fit):
    labels = ['Temperature (C)', 'Total Rain (mm)', 'Total Snow (cm)',
         'Hour', 'Weekday', 'Month']
    dot_data = tree.export_graphviz(fit, out_file=None, 
                                    max_depth=3, filled=True, 
                                    rounded=True,special_characters=True
                                , feature_names = labels, impurity=False)

    graph = pydotplus.graph_from_dot_data(dot_data)

    colors = ('lightskyblue','mediumpurple1')
    edges = collections.defaultdict(list)

    for edge in graph.get_edge_list():
        edges[edge.get_source()].append(int(edge.get_destination()))

    for edge in edges:
        edges[edge].sort()    
        for i in range(2):
            dest = graph.get_node(str(edges[edge][i]))[0]
            dest.set_fillcolor(colors[i])
    return graph

In [95]:
# parameter tests
parameters = {'min_samples_leaf': [10,50,100,250,500,750,1000,
                                   1250,1500,1750,2000]}
# natural data training fit
start_training = time.perf_counter()
dec_tree_nat = DecisionTreeRegressor()
fit_nat = GridSearchCV(dec_tree_nat, parameters, cv=5, refit = True)
fit_nat.fit(x_train_nat, y_train)
tree_predict_nat = fit_nat.predict(x_test_nat)

# Best min_samples_leaf Parameter
opt_nat = fit_nat.best_params_

# Test Set fit
tree_nat = DecisionTreeRegressor(min_samples_leaf = 10)
tree_nat.fit(x_train_nat, y_train)
tree_predict_nat = tree_nat.predict(x_test_nat)
end_training = time.perf_counter()

# Plot Tree
graph = treePlot(tree_nat)
graph.write_png('tree_natural.png')

print('Computing time to fit Decision Tree using Natural Data : ', end_training - start_training,' seconds')
print('Best min_samples_leaf Parameter: ', opt_nat)
print('Natural Data RMSE: ', rmse(tree_predict_nat, y_test))
print('Natural Data Mean Absolute Percent Error: ', 
      percent_error(tree_predict_nat, y_test))

Time Elapsed to fit Decision Tree Natural Data :  33.936360104000414  seconds
Best min_samples_leaf Parameter:  {'min_samples_leaf': 10}
Natural Data RMSE:  0.10561179860289971
Natural Data Mean Absolute Percent Error:  0.07930057286123038


In [96]:
# Engineered data training fit
start_training = time.perf_counter()
dec_tree_eng = DecisionTreeRegressor()
fit_eng = GridSearchCV(dec_tree_eng, parameters, cv=5, refit = True)
fit_eng.fit(x_train_eng, y_train)
tree_predict_eng = fit_eng.predict(x_test_eng)

# Best min_samples_leaf Parameter
opt_eng = fit_eng.best_params_

# Test Set fit
tree_eng = DecisionTreeRegressor(min_samples_leaf = 10)
tree_eng.fit(x_train_eng, y_train)
tree_predict_eng = tree_eng.predict(x_test_eng)
end_training = time.perf_counter()

# Plot Tree
graph = treePlot(tree_eng)
graph.write_png('tree_engineered.png')

print('Computing time to fit Decision Tree using Engineered Data : ', end_training - start_training,' seconds')
print('Best min_samples_leaf Parameter: ', opt_eng)
print('Engineered Data RMSE: ', rmse(tree_predict_eng, y_test))
print('Engineered Data Mean Absolute Percent Error: ', 
      percent_error(tree_predict_eng, y_test))

Time Elapsed to fit Decision Tree :  23.00414419400022  seconds
Best min_samples_leaf Parameter:  {'min_samples_leaf': 10}
Engineered Data RMSE:  0.2891866545576817
Engineered Data Mean Absolute Percent Error:  0.2069902620594885
